In [1]:
import igl
import pybcclean as pbc
import polyfempy as pf
import wildmeshing as wm
import meshplot as mp
import os
import numpy as np
from scipy.spatial import KDTree
import yaml
def barycenters(v, f):
    s = np.zeros((len(f),3),dtype='float64')
    for i, r in enumerate(s):
        s[i] = (v[f[i][0]]+v[f[i][1]]+v[f[i][2]])/3
    return s

def parse_feat(in_feat_file_name, out_file_body="feat"):
    parent_dir = os.path.dirname(in_feat_file_name)
    out_file_name = out_file_body+".dmat"
    out_file_rel_dir = os.path.join(parent_dir, out_file_name)
        # return the file path if the output file already exists
    with open(in_feat_file_name, 'r') as stream:
        try:
            yaml_dict=yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    face_label_dict = {}
    count =0
    for label, surface in enumerate(yaml_dict["surfaces"]):
        for face_id in surface["face_indices"]:
            face_label_dict[face_id] = label
            count +=1

    with open(out_file_rel_dir, 'w') as f:
        f.write("1"+" "+str(count)+"\n")
        for item in sorted(face_label_dict.keys()):
            f.write("%s\n" % face_label_dict[item])
    return out_file_rel_dir

def separate_surfaces(v, f, fl):
    """
    :param v vertices
    :param f faces
    :param fl face labels
    perturb the surfaces according to the face labels
    :return v_dict, f_dict
    """
    v_dict = {}
    f_dict_temp = {}
    f_dict = {}
    count_dict = {}
    for fidx, lb in enumerate(fl):
        if lb not in f_dict_temp:
            f_dict_temp[lb] = np.zeros_like(f)
            f_dict_temp[lb][0,:]=f[fidx,:].copy()
            count_dict[lb] = 1
        else:
            f_dict_temp[lb][count_dict[lb],:]=f[fidx,:].copy()
            count_dict[lb] += 1
    for lb in f_dict_temp:
        f_dict_temp[lb] = f_dict_temp[lb][0:count_dict[lb],:]
        v_dict[lb], f_dict[lb], _, _= igl.remove_unreferenced(v,f_dict_temp[lb])
    return v_dict, f_dict

def perturb_and_union(v_dict, f_dict, eps):
    """
    :eps float to control the perturbation
    return new_v, new_f, new_fl
    """
    nv = 0 
    nf = 0
    for lb in f_dict:
        nv += len(v_dict[lb])
        nf += len(f_dict[lb])
    new_v = np.zeros((nv,3),dtype=float)
    new_f = np.zeros((nf,3), dtype=int)
    new_fl = np.zeros((nf,1), dtype=int)
    count_v = 0
    count_f =0
    for lb in f_dict:
        f_temp = f_dict[lb].copy()
        v_temp = v_dict[lb].copy()
        new_v[count_v:count_v+len(v_temp)]= v_temp + np.ones_like(v_temp) * np.random.uniform(-eps, eps)
        f_temp += np.ones_like(f_temp) * count_v
        count_v += len(v_temp)
        new_f[count_f:count_f+len(f_temp)]=f_temp
        new_fl[count_f:count_f+len(f_temp)]= np.ones((len(f_temp),1), dtype=int) *lb
        count_f += len(f_temp)
    return new_v, new_f, new_fl

def extract_surface(p, t):
    f_temp = igl.boundary_facets(t)
    v_s, f_s, _, _ = igl.remove_unreferenced(p, f_temp)
    return v_s, f_s

file_root = os.path.dirname("14/00000022_ad34a3f60c4a4caa99646600_trimesh_002.obj")
# wm.tetrahedralize("14/00000022_ad34a3f60c4a4caa99646600_trimesh_002.obj", file_root+"/"+"bench.mesh", stop_quality=7)
fl_bench_file = parse_feat("14/00000022_ad34a3f60c4a4caa99646600_features_002.yml")
fl_bench = igl.read_dmat(fl_bench_file)
v_bench, f_bench = igl.read_triangle_mesh("14/00000022_ad34a3f60c4a4caa99646600_trimesh_002.obj")
v_ini, f_ini=igl.read_triangle_mesh(file_root+"/"+"bench.mesh__sf.obj")
prob_mat_ini, fl_ini_temp =pbc.project_face_labels(v_bench,f_bench.astype('int32'), fl_bench.astype('int32'),v_ini,f_ini.astype('int32'))
fl_ini = pbc.refine_labels(v_ini, f_ini.astype('int32'), prob_mat_ini, fl_ini_temp.astype('int32'),1)
# mp.plot(v_ini, f_ini, fl_ini_temp[:,0])
# mp.plot(v_ini, f_ini, fl_ini[:,0])
eps = 0.001
v_dict, f_dict =separate_surfaces(v_ini, f_ini, fl_ini[:,0])
v_bad, f_bad, fl_bad = perturb_and_union(v_dict, f_dict, eps)
mp.plot(v_bad, f_bad, fl_bad[:,0], shading={"wireframe":True})
eps_file_name = file_root+"/"+str(eps)+"pertb.obj"
igl.write_obj(eps_file_name, v_bad, f_bad)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(12.499931…

True

In [2]:
wm.tetrahedralize(eps_file_name, file_root+"/"+"out.mesh", stop_quality=7)
v_good, f_good = igl.read_triangle_mesh(file_root+"/"+"out.mesh__sf.obj")
print(f_good.shape, v_good.shape)
mp.plot(v_good, f_good, shading={"wireframe":True})

(1708, 3) (856, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(12.500144…

In [3]:
prob_mat, fl_good_proj = pbc.project_face_labels(v_bad, f_bad.astype('int32'), fl_bad[:,0].astype('int32'), v_good, f_good.astype('int32'))
fl_good_cut = pbc.refine_labels(v_good, f_good.astype('int32'), prob_mat, fl_good_proj.astype('int32'),1)
mp.plot(v_good, f_good, fl_good_proj[:,0], shading={"wireframe": True})
mp.plot(v_good, f_good, fl_good_cut[:,0], shading={"wireframe":True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(12.500144…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(12.500144…

In [4]:
bary_centers = barycenters(v_good, f_good)
kdt = KDTree(bary_centers)

bary_centers_ini = barycenters(v_ini, f_ini)
kdt_ini = KDTree(bary_centers_ini)

def sideset_proj(p):
    return fl_good_proj[kdt.query(p)[1]][0]+1
def sideset_cut(p):
    return fl_good_cut[kdt.query(p)[1]][0] + 1
def sideset_ini(p):
    return fl_ini[kdt_ini.query(p)[1]][0] + 1

solver_ini = pf.Solver()
solver_ini.load_mesh_from_path(file_root+"/"+"bench.mesh")
solver_ini.set_boundary_side_set_from_bary(sideset_ini)
p_ini, t_ini, s_ini = solver_ini.get_boundary_sidesets()
mp.plot(p_ini, t_ini, s_ini, shading={"wireframe":True})

solver_proj = pf.Solver()
solver_proj.load_mesh_from_path(file_root+"/"+"out.mesh")
solver_proj.set_boundary_side_set_from_bary(sideset_proj)
p_proj, t_proj, s_proj = solver_proj.get_boundary_sidesets()
mp.plot(p_proj, t_proj, s_proj, shading={"wireframe":True})

solver_cut = pf.Solver()
solver_cut.load_mesh_from_path(file_root+"/"+"out.mesh")
solver_cut.set_boundary_side_set_from_bary(sideset_cut)
p_cut, t_cut, s_cut = solver_cut.get_boundary_sidesets()
mp.plot(p_cut, t_cut, s_cut, shading={"wireframe":True})


[2019-08-29 09:47:57.585] [polyfem] [info] Loading mesh...
[2019-08-29 09:47:57.588] [geogram] [info] Loading file 14/bench.mesh...
[2019-08-29 09:47:57.639] [geogram] [info] (FP64) nb_v:2830 nb_e:0 nb_f:1888 nb_b:0 tri:1 dim:3
[2019-08-29 09:47:57.639] [geogram] [info]  nb_tets:13698
[2019-08-29 09:47:57.639] [geogram] [info] Attributes on vertices: point[3]
[2019-08-29 09:47:57.818] [polyfem] [info] mesh bb min [-4.44089e-16, -25, -1.77636e-15], max [25, 8.88178e-16, 25]
[2019-08-29 09:47:57.819] [polyfem] [info]  took 0.233371s


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(12.5, -12…

[2019-08-29 09:47:58.225] [polyfem] [info] Loading mesh...
[2019-08-29 09:47:58.225] [geogram] [info] Loading file 14/out.mesh...
[2019-08-29 09:47:58.287] [geogram] [info] (FP64) nb_v:2646 nb_e:0 nb_f:1708 nb_b:0 tri:1 dim:3
[2019-08-29 09:47:58.287] [geogram] [info]  nb_tets:12910
[2019-08-29 09:47:58.287] [geogram] [info] Attributes on vertices: point[3]
[2019-08-29 09:47:58.579] [polyfem] [info] mesh bb min [-5.55358e-05, -25.0006, -0.00130673], max [25.0003, 0.000668736, 25.0007]
[2019-08-29 09:47:58.579] [polyfem] [info]  took 0.354349s


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(12.500144…

[2019-08-29 09:47:58.960] [polyfem] [info] Loading mesh...
[2019-08-29 09:47:58.960] [geogram] [info] Loading file 14/out.mesh...
[2019-08-29 09:47:59.007] [geogram] [info] (FP64) nb_v:2646 nb_e:0 nb_f:1708 nb_b:0 tri:1 dim:3
[2019-08-29 09:47:59.007] [geogram] [info]  nb_tets:12910
[2019-08-29 09:47:59.007] [geogram] [info] Attributes on vertices: point[3]
[2019-08-29 09:47:59.293] [polyfem] [info] mesh bb min [-5.55358e-05, -25.0006, -0.00130673], max [25.0003, 0.000668736, 25.0007]
[2019-08-29 09:47:59.293] [polyfem] [info]  took 0.333005s


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(12.500144…

In [5]:
settings = pf.Settings()
problem = pf.Problem()

settings.set_pde(pf.PDEs.LinearElasticity)

settings.set_material_params("E", 200)
settings.set_material_params("nu", 0.35)


problem.set_displacement(2, [0, 0, 0])
problem.set_force(3, [0, 50, 0])

settings.set_problem(problem)
solver_proj.settings(settings)
solver_proj.solve()
p, t, d = solver_proj.get_sampled_solution()
m = np.linalg.norm(d, axis=1)

print(p.shape)
print(t.shape)
print(d.shape)
mp.plot(p+d, t, m)

[2019-08-29 09:48:05.301] [polyfem] [info] simplex_count: 	12910
[2019-08-29 09:48:05.301] [polyfem] [info] regular_count: 	0
[2019-08-29 09:48:05.301] [polyfem] [info] regular_boundary_count: 	0
[2019-08-29 09:48:05.301] [polyfem] [info] simple_singular_count: 	0
[2019-08-29 09:48:05.301] [polyfem] [info] multi_singular_count: 	0
[2019-08-29 09:48:05.301] [polyfem] [info] boundary_count: 	0
[2019-08-29 09:48:05.301] [polyfem] [info] multi_singular_boundary_count: 	0
[2019-08-29 09:48:05.301] [polyfem] [info] non_regular_count: 	0
[2019-08-29 09:48:05.301] [polyfem] [info] non_regular_boundary_count: 	0
[2019-08-29 09:48:05.301] [polyfem] [info] undefined_count: 	0
[2019-08-29 09:48:05.301] [polyfem] [info] total count:	 12910
[2019-08-29 09:48:05.302] [polyfem] [info] Building isoparametric basis...
[2019-08-29 09:48:05.371] [polyfem] [info] Computing polygonal basis...
[2019-08-29 09:48:05.372] [polyfem] [info]  took 0.000552146s
[2019-08-29 09:48:05.373] [polyfem] [info] hmin: 0.921

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(12.758327…

In [6]:
solver_cut.settings(settings)
solver_cut.solve()
p1, t1, d1 = solver_cut.get_sampled_solution()
m1 = np.linalg.norm(d1, axis=1)


mp.plot(p1+d1, t1, m1)

[2019-08-29 09:48:22.098] [polyfem] [info] simplex_count: 	12910
[2019-08-29 09:48:22.098] [polyfem] [info] regular_count: 	0
[2019-08-29 09:48:22.098] [polyfem] [info] regular_boundary_count: 	0
[2019-08-29 09:48:22.098] [polyfem] [info] simple_singular_count: 	0
[2019-08-29 09:48:22.098] [polyfem] [info] multi_singular_count: 	0
[2019-08-29 09:48:22.098] [polyfem] [info] boundary_count: 	0
[2019-08-29 09:48:22.098] [polyfem] [info] multi_singular_boundary_count: 	0
[2019-08-29 09:48:22.098] [polyfem] [info] non_regular_count: 	0
[2019-08-29 09:48:22.098] [polyfem] [info] non_regular_boundary_count: 	0
[2019-08-29 09:48:22.098] [polyfem] [info] undefined_count: 	0
[2019-08-29 09:48:22.098] [polyfem] [info] total count:	 12910
[2019-08-29 09:48:22.098] [polyfem] [info] Building isoparametric basis...
[2019-08-29 09:48:22.159] [polyfem] [info] Computing polygonal basis...
[2019-08-29 09:48:22.159] [polyfem] [info]  took 5.6311e-05s
[2019-08-29 09:48:22.165] [polyfem] [info] hmin: 0.9219

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(12.465914…

In [7]:
solver_ini.settings(settings)
solver_ini.solve()
p2, t2, d2 = solver_ini.get_sampled_solution()
m2 = np.linalg.norm(d2, axis=1)


mp.plot(p2+d2, t2, m2)

[2019-08-29 09:48:31.548] [polyfem] [info] simplex_count: 	13698
[2019-08-29 09:48:31.548] [polyfem] [info] regular_count: 	0
[2019-08-29 09:48:31.548] [polyfem] [info] regular_boundary_count: 	0
[2019-08-29 09:48:31.548] [polyfem] [info] simple_singular_count: 	0
[2019-08-29 09:48:31.548] [polyfem] [info] multi_singular_count: 	0
[2019-08-29 09:48:31.548] [polyfem] [info] boundary_count: 	0
[2019-08-29 09:48:31.548] [polyfem] [info] multi_singular_boundary_count: 	0
[2019-08-29 09:48:31.548] [polyfem] [info] non_regular_count: 	0
[2019-08-29 09:48:31.548] [polyfem] [info] non_regular_boundary_count: 	0
[2019-08-29 09:48:31.548] [polyfem] [info] undefined_count: 	0
[2019-08-29 09:48:31.548] [polyfem] [info] total count:	 13698
[2019-08-29 09:48:31.548] [polyfem] [info] Building isoparametric basis...
[2019-08-29 09:48:31.627] [polyfem] [info] Computing polygonal basis...
[2019-08-29 09:48:31.627] [polyfem] [info]  took 5.0693e-05s
[2019-08-29 09:48:31.629] [polyfem] [info] hmin: 0.9830

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(12.580124…

In [8]:
v_1, f_1 = extract_surface(p1+d1, t1)
igl.write_obj("1.obj", v_1, f_1)
v_1, f_1 = igl.read_triangle_mesh("1.obj")
v_2, f_2 = extract_surface(p2+d2, t2)
igl.write_obj("2.obj", v_2, f_2)
v_2, f_2 = igl.read_triangle_mesh("2.obj")
v_, f_ = extract_surface(p+d, t)
igl.write_obj("_.obj", v_, f_)
v_, f_ = igl.read_triangle_mesh("_.obj")
print(igl.hausdorff(v_1, f_1, v_2, f_2))
print(igl.hausdorff(v_, f_, v_2, f_2))

0.5732265489152076
3.104443169054964


In [ ]:
file_root1 = os.path.dirname("3/00000008_9b3d6a97e8de4aa193b81000_trimesh_000.obj")
wm.tetrahedralize("3/00000008_9b3d6a97e8de4aa193b81000_trimesh_000.obj", file_root1+"/"+"bench.mesh", stop_quality=7)
fl_bench_file1 = parse_feat("3/00000008_9b3d6a97e8de4aa193b81000_features_000.yml")
fl_bench1 = igl.read_dmat(fl_bench_file1)
v_bench1, f_bench1 = igl.read_triangle_mesh("3/00000008_9b3d6a97e8de4aa193b81000_trimesh_000.obj")
v_ini1, f_ini1=igl.read_triangle_mesh(file_root1+"/"+"bench.mesh__sf.obj")
prob_mat_ini1, fl_ini_temp1 =pbc.project_face_labels(v_bench1,f_bench1.astype('int32'), fl_bench1.astype('int32'),v_ini1,f_ini1.astype('int32'))
fl_ini1 = pbc.refine_labels(v_ini1, f_ini1.astype('int32'), prob_mat_ini1, fl_ini_temp1.astype('int32'),1)
# mp.plot(v_ini, f_ini, fl_ini_temp[:,0])
# mp.plot(v_ini, f_ini, fl_ini[:,0])
eps = 0.001
v_dict1, f_dict1 =separate_surfaces(v_ini1, f_ini1, fl_ini1[:,0])
v_bad1, f_bad1, fl_bad1 = perturb_and_union(v_dict1, f_dict1, eps)
mp.plot(v_bad1, f_bad1, fl_bad1[:,0], shading={"wireframe":True})
eps_file_name1 = file_root1+"/"+str(eps)+"pertb.obj"
igl.write_obj(eps_file_name1, v_bad1, f_bad1)

In [4]:
wm.tetrahedralize(eps_file_name1, "test.mesh", stop_quality=7)
v_good1, f_good1 = igl.read_triangle_mesh(file_root1+"/"+"out.mesh__sf.obj")
print(f_good1.shape, v_good1.shape)
mp.plot(v_good1, f_good1, shading={"wireframe":True})

(2464, 3) (1232, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000126…

In [ ]:
prob_mat1, fl_good_proj1 = pbc.project_face_labels(v_bad1, f_bad1.astype('int32'), fl_bad1[:,0].astype('int32'), v_good1, f_good1.astype('int32'))
# fl_good_cut1 = pbc.refine_labels(v_good1, f_good1.astype('int32'), prob_mat1, fl_good_proj1.astype('int32'),1)
# mp.plot(v_good1, f_good1, fl_good_proj1[:,0], shading={"wireframe": True})
# mp.plot(v_good1, f_good1, fl_good_cut1[:,0], shading={"wireframe":True})

In [ ]:
v_test, f_test=igl.upsample(v_good,f_good)
